<a href="https://colab.research.google.com/github/shazzad-hasan/few-shot-learning/blob/main/siamese_neural_network/siamese_omniglot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/shazzad-hasan/few-shot-learning.git

Cloning into 'few-shot-learning'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 52 (delta 6), reused 16 (delta 0), pack-reused 0
Unpacking objects: 100% (52/52), done.


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!ls

drive  few-shot-learning  sample_data


In [4]:
%cd /content/few-shot-learning/siamese_neural_network

/content/few-shot-learning/siamese_neural_network


In [5]:
!ls

helper_dataset.py  helper_evaluate.py  helper_train.py	siamese_omniglot.ipynb


In [6]:
# import required libraries
import torch
import torchvision 

import numpy as np
import matplotlib.pyplot as plt

In [7]:
# import local helper functions
from helper_dataset import dataloader_omniglot
from helper_train import train
from helper_evaluate import test

SyntaxError: ignored

In [ ]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
  print("CUDA is available!")
else:
  print("CUDA is not available")

device = torch.device('cuda') if train_on_gpu else torch.device('cpu')

In [ ]:
train_loader, valid_loader, test_loader = dataloader_omniglot(train_size, valid_pct,
    test_size, batch_size, n_way, num_workers, valid_size)

In [ ]:
for img1, img2, label in train_loader:
  if label[0] == 1.0:
    plt.subplot(1,2,1)
    plt.imshow(img1[0][0])
    plt.subplot(1,2,2)
    plt.imshow(img2[0][0])
    break

In [ ]:
# print out some data stats
print("Number of training images: ", len(train_data))
print("Number of validation images: ", len(valid_data))
print("Number of test images: ", len(test_data))

for img, _, _ in train_loader:
  print("Image batch dimension: ", img.shape)
  break

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SiameseNetwork(nn.Module):
  def __init__(self):
    super(SiameseNetwork, self).__init__()
    
    # Conv2d(in_channels, out_channels, kernel_size)
    self.conv1 = nn.Conv2d(1, 64, 10)
    self.conv2 = nn.Conv2d(64, 128, 7)
    self.conv3 = nn.Conv2d(128, 128, 4)
    self.conv4 = nn.Conv2d(128, 256, 4)

    self.bn1 = nn.BatchNorm2d(64)
    self.bn2 = nn.BatchNorm2d(128)
    self.bn3 = nn.BatchNorm2d(128)
    self.bn4 = nn.BatchNorm2d(256)

    self.dropout1 = nn.Dropout(0.1)
    self.dropout2 = nn.Dropout(0.5)

    self.fc1 = nn.Linear(256*6*6, 4096)
    self.fc2 = nn.Linear(4096, 1)

  def convolution(self, x):
    x = F.relu(self.bn1(self.conv1(x)))
    x = F.max_pool2d(x, (2,2))
    x = F.relu(self.bn2(self.conv2(x)))
    x = F.max_pool2d(x, (2,2))
    x = F.relu(self.bn3(self.conv3(x)))
    x = F.max_pool2d(x, (2,2))
    x = F.relu(self.bn4(self.conv4(x)))

    return x

  def forward(self, x1, x2):
    x1 = self.convolution(x1)
    x2 = self.convolution(x2)

    # flatten input image
    x1 = x1.view(-1, 256*6*6)
    x2 = x2.view(-1, 256*6*6)

    x1 = torch.sigmoid(self.fc1(x1))
    x2 = torch.sigmoid(self.fc1(x2))

    x = torch.abs(x1 - x2)
    x = self.fc2(x)

    return x

model = SiameseNetwork()
model.to(device)
print(model)

In [ ]:
import torch.optim as optim

# specify a loss function
criterion = nn.BCEWithLogitsLoss() # categorical cross-entropy

# specify optimizer
params = model.parameters()
optimizer = optim.Adam(params, lr=0.001)

In [ ]:
train_losses, valid_losses = train(model, train_loader, valid_loader, num_epochs, criterion, optimizer, device)